# Overview of Content:


### mlo_01_import:


__Sections__:
- Main Module Import Section and Notebook Settings
- Initialize Custom Functions
- Import / Download Data
- Create Subfolders if they do not exists
- Mauna Loa Data (monthly mean)
- ONI-Data (Oceanic Nino Index)


__Save Dataframes as Notebook Variables and as Text Files__:
- Mauna Loa Data (monthly mean):
    - mmlo
    - eq_trend_mmlo
    - p_trend_mmlo
- ONI-Data (Oceanic Nino Index):
    - oeni
    - eq_trend_oeni

## Main Module Import Section and Notebook Settings

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date
from datetime import timedelta
import time
from scipy.fftpack import fft
from scipy import signal
import math
import matplotlib.pyplot as plt
%matplotlib notebook

plt.style.use('ggplot')
from matplotlib import rcParams
rcParams['lines.linewidth'] = 0.9
rcParams['figure.figsize'] = 9, 5
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%config InlineBackend.figure_format = 'retina'


myred = '#E24A33'
myblue = '#348ABD'


## Initialize Custom Functions

In [2]:
def calc_trend(dec_date, data, deg=2):
    """
    calculates a polynomial fit with least square methods
    """
    p = np
    p = np.polyfit(dec_date, data, deg)
    
    if deg == 1:
        trend = (p[0] * dec_date) + p[1]
        equation = str(np.round(p[0], 5)) + ' x + ' + str(np.round(p[1], 3))
    
    elif deg == 2:
        trend = (p[0] * dec_date**2) + (p[1] * dec_date) + p[2]
        equation = str(np.round(p[0], 3)) + ' x^2 + ' + str(np.round(p[1], 3)) + ' x + ' + str(np.round(p[2], 3))
        
    elif deg == 3:
        trend = (p[0] * dec_date**3) + (p[1] * dec_date**2) + (p[2] * dec_date) + p[3]
        equation = str(np.round(p[0], 3)) + ' x^3 + ' + str(np.round(p[1], 3)) + \
        ' x^2 + ' + str(np.round(p[2], 3)) + ' x + ' + str(np.round(p[3], 3))
                                                                                                                                                  
    return trend, equation, p

In [3]:
def create_dir_os(mydir):
    """
    create directory if it does not exist
    """
    if not os.path.exists(mydir):
        os.makedirs(mydir)

In [4]:
def yr_fraction(t_date):
    """
    returns a datetime as decimal years
    """
    
    if np.size(t_date) == 1:
        t_dates = []
        t_dates.append(t_date)
    else:
        t_dates = t_date
        
    date_list = []
    for i in t_dates:
        def sinceEpoch(i): # returns seconds since epoch
            return time.mktime(i.timetuple())

        s = sinceEpoch

        year = i.year
        startOfThisYear = datetime(year=year, month=1, day=1)
        startOfNextYear = datetime(year=year+1, month=1, day=1)

        yearElapsed = s(i) - s(startOfThisYear)
        yearDuration = s(startOfNextYear) - s(startOfThisYear)
        fraction = yearElapsed/yearDuration
        date_list.append(i.year + fraction)
    
    return date_list

# Import / Download Data

*choose if new online data shoud be downloaded (old data files will be replaced)*

In [5]:
# True: download new files
# False: use local files
download_online_mmlo = True  # monthly mean mauna loa data
download_online_oeni = True  # ocean nino index data

## Create Subfolders if they do not exists

In [6]:
create_dir_os('data')  # create folder for downloaded time series
create_dir_os('data_results')  # create folder for exported variables as text-files
create_dir_os('plots')  # create folder for figures / plots

### Mauna Loa Data (monthly mean)

Data from: ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_mm_mlo.txt


In [7]:
# download data, save it as txt file in folder /data and read it as pandas table
if download_online_mmlo == True:
    online_mmlo = 'ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_mm_mlo.txt'
    path_mmlo = 'data/co2_mm_mlo.txt'
    data_online = pd.read_table(online_mmlo)
    data_online.to_csv(path_mmlo, index=False)

In [8]:
# read time series as pandas dataframe
path_mmlo = 'data/co2_mm_mlo.txt'
# read data as pandas table (dataframe), replace missing data with NaN
data_raw = pd.read_table(path_mmlo, header=None, skiprows=72, delim_whitespace=True,
                     names=['year', 'month', 'dec_date', 'monthly_avg', 'monthly_interpolated', 'trend_season', 'days'],
                     index_col=[0, 1], na_values='-99.99')
data_raw.head()
data_raw.tail()

dec_date  monthly_avg  monthly_interpolated  trend_season  days
year month                                                                 
1958 3      1958.208       315.71                315.71        314.62    -1
     4      1958.292       317.45                317.45        315.29    -1
     5      1958.375       317.50                317.50        314.71    -1
     6      1958.458          NaN                317.10        314.85    -1
     7      1958.542       315.86                315.86        314.98    -1

dec_date  monthly_avg  monthly_interpolated  trend_season  days
year month                                                                 
2017 2      2017.125       406.42                406.42        405.61    26
     3      2017.208       407.18                407.18        405.61    23
     4      2017.292       409.00                409.00        406.11    25
     5      2017.375       409.65                409.65        406.26    27
     6      2017.458       408.84                408.84        406.56    26

In [9]:
# create a datetime index via pandas in a monthly frequency (last day of month)
index_data = pd.date_range(date(data_raw.index.get_level_values(0)[0],
                                data_raw.index.get_level_values(1)[0], 1),
                                periods=data_raw.index.size, freq='M')
data = data_raw.set_index(index_data).copy()

#### Save in New Data Format (mmlo)

In [10]:
mmlo = pd.DataFrame()
mmlo['mmlo_data'] = data.monthly_interpolated  # interpolated data from NOAA
mmlo['mmlo_dec_date'] = yr_fraction(mmlo.index)  # calculate decimal annual date from new datetime
mmlo['mmlo_sea'] = data.trend_season  # monthly seasonal corrected trend data from NOAA


# get trend and its parameters from function
mmlo['mmlo_trend'], eq_trend_mmlo, p_trend_mmlo = calc_trend(mmlo.mmlo_dec_date.values, 
                                                mmlo.mmlo_data.values)  # calculate polyomial fit
mmlo['mmlo_notrend'] = mmlo.mmlo_data - mmlo.mmlo_trend  # detrend data with fit
mmlo.head()
mmlo.tail()

# save internal jupyter notebook variable for access from other notebooks
%store mmlo
%store eq_trend_mmlo
%store p_trend_mmlo

# save dataframes as text file
%store mmlo >data_results/mmlo.txt
%store eq_trend_mmlo >data_results/eq_trend_mmlo.txt


,mmlo_data,mmlo_dec_date,mmlo_sea,mmlo_trend,mmlo_notrend
1958-03-31,315.71,1958.243836,314.62,314.438050,1.271950
1958-04-30,317.45,1958.326027,315.29,314.502831,2.947169
1958-05-31,317.50,1958.410959,314.71,314.569951,2.930049
1958-06-30,317.10,1958.493151,314.85,314.635079,2.464921
1958-07-31,315.86,1958.578082,314.98,314.702556,1.157444


,mmlo_data,mmlo_dec_date,mmlo_sea,mmlo_trend,mmlo_notrend
2017-02-28,406.42,2017.158904,405.61,404.529537,1.890463
2017-03-31,407.18,2017.243721,405.61,404.722266,2.457734
2017-04-30,409.00,2017.325913,406.11,404.909201,4.090799
2017-05-31,409.65,2017.410845,406.26,405.102547,4.547453
2017-06-30,408.84,2017.493037,406.56,405.289828,3.550172


Stored 'mmlo' (DataFrame)
Stored 'eq_trend_mmlo' (str)
Stored 'p_trend_mmlo' (ndarray)
Writing 'mmlo' (DataFrame) to file 'data_results/mmlo.txt'.
Writing 'eq_trend_mmlo' (str) to file 'data_results/eq_trend_mmlo.txt'.


### ONI-Data (Oceanic Nino Index)

Data from: http://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt

#### SAME PROCEDURE AS WITH MONTHLY MAUNA LOA DATA:

In [11]:
# download and save data
if download_online_oeni == True:
    online_oeni = 'http://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt'
    path_oeni = 'data/oni.ascii.txt'
    data_online = pd.read_table(online_oeni)
    data_online.to_csv(path_oeni, index=False)

In [12]:
# read time series as pandas dataframe
path_oeni = 'data/oni.ascii.txt'

oeni_raw = pd.read_table(path_oeni, header=None, skiprows=1, delim_whitespace=True,
                    names=['season', 'year', 'total', 'anomaly'],
                    index_col=[1])
oeni_raw.head()
oeni_raw.tail()

,season,total,anomaly
year,,,
1950,DJF,25.01,-1.38
1950,JFM,25.42,-1.21
1950,FMA,25.92,-1.11
1950,MAM,26.25,-1.16
1950,AMJ,26.42,-1.11


,season,total,anomaly
year,,,
2017,DJF,26.32,-0.42
2017,JFM,26.79,-0.12
2017,FMA,27.47,0.15
2017,MAM,28.11,0.39
2017,AMJ,28.35,0.48


In [13]:
# create a datetime index via pandas in a monthly frequency (last day of month)
index_oeni = pd.date_range(date(oeni_raw.index[0], 1, 1),
                          periods=oeni_raw.index.size, freq='M')
oeni_tmp = oeni_raw.set_index(index_oeni).copy()

#### Save in New Data Format (oeni)

In [14]:
oeni = pd.DataFrame()
oeni['oeni_data'] = oeni_tmp.anomaly  # SST anomaly data
oeni['oeni_dec_date'] = yr_fraction(oeni_tmp.index)
oeni['oeni_trend'], eq_trend_oeni, p_trend_oeni = calc_trend(oeni.oeni_dec_date.values, oeni.oeni_data.values, deg=1)
oeni['oeni_notrend'] = oeni.oeni_data - oeni.oeni_trend
oeni.head()
oeni.tail()

# save internal jupyter notebook variable for access from other notebooks
%store oeni
%store eq_trend_oeni

# save dataframes as text file
%store oeni >data_results/oeni.txt
%store eq_trend_oeni >data_results/eq_trend_oeni.txt

,oeni_data,oeni_dec_date,oeni_trend,oeni_notrend
1950-01-31,-1.38,1950.082192,0.015703,-1.395703
1950-02-28,-1.21,1950.158904,0.015713,-1.225713
1950-03-31,-1.11,1950.243836,0.015724,-1.125724
1950-04-30,-1.16,1950.326027,0.015734,-1.175734
1950-05-31,-1.11,1950.410959,0.015745,-1.125745


,oeni_data,oeni_dec_date,oeni_trend,oeni_notrend
2017-01-31,-0.42,2017.082192,0.024230,-0.444230
2017-02-28,-0.12,2017.158904,0.024240,-0.144240
2017-03-31,0.15,2017.243721,0.024251,0.125749
2017-04-30,0.39,2017.325913,0.024261,0.365739
2017-05-31,0.48,2017.410845,0.024272,0.455728


Stored 'oeni' (DataFrame)
Stored 'eq_trend_oeni' (str)
Writing 'oeni' (DataFrame) to file 'data_results/oeni.txt'.
Writing 'eq_trend_oeni' (str) to file 'data_results/eq_trend_oeni.txt'.
